Learn to add two 2-digit numbers with fixed-size padded at the right blocks.

increasing tropout seems to help


In [1]:
from gptbench import Sample, LogFlag, Train, empty_config

To create train and validation dataset:
python prepare_addition.py ../data/add2.txt 2 --sep="\n" --split=0.9

Creates add2.train.txt and add2.val.txt

In [2]:
with open('../data/add2.val.txt', 'r', newline=None) as f:
    val_data = f.read()
val_data[:500]

'90+0=90\n90+1=91\n90+2=92\n90+3=93\n90+4=94\n90+5=95\n90+6=96\n90+7=97\n90+8=98\n90+9=99\n90+10=100\n90+11=101\n90+12=102\n90+13=103\n90+14=104\n90+15=105\n90+16=106\n90+17=107\n90+18=108\n90+19=109\n90+20=110\n90+21=111\n90+22=112\n90+23=113\n90+24=114\n90+25=115\n90+26=116\n90+27=117\n90+28=118\n90+29=119\n90+30=120\n90+31=121\n90+32=122\n90+33=123\n90+34=124\n90+35=125\n90+36=126\n90+37=127\n90+38=128\n90+39=129\n90+40=130\n90+41=131\n90+42=132\n90+43=133\n90+44=134\n90+45=135\n90+46=136\n90+47=137\n90+48=138\n90+49=139\n90+50=140\n90+51=141\n'

In [3]:
ben = Train('add2pad', log_mask=LogFlag.ALL)

# set datasets
ben.set_datasets('padlinechar', train_path='../data/add2.train.txt', val_path='../data/add2.val.txt')

# set config settings
cfg = empty_config()
cfg.train.log_period=0
cfg.model.set(n_layer=6, n_head=6, n_embd=90, block_size=16)
cfg.sample.set(top=1, max_batch_size=256) # top_k(1) - always pick the best item
cfg.train.set(sample_period=-5)
cfg.trainer.set(batch_size=128)

# and init a new model with config
if ben.can_resume():
    ben.init_resume(cfg)
else:
    ben.init_new(cfg)
# print(do.get_config().dump(1))

New random seed 768391787
Loading checkpoint from ./models/add2pad/
Checkpoint: iter=1900 (27.022 epoch), loss train=0.4691 val=0.7730 eval->0.7730
Dataset train_path: ../data/add2.train.txt, val_path: ../data/add2.val.txt, train_split: None, vocab_size: 13
Model params: 0.59M


In [4]:
#ben.line_dataset_split_qa(ben.val_dataset, 0, 20, sep='=')

In [15]:
ds = ben.val_dataset
q,a=ds.sample_split(0, len(ds), sep='=', sep_included=-1)

def test(q,a,g):
    res = float(a == g)
    if not res:
        print(f"{q}: {a} != {g}")
    return res
    
ben.measure_accuracy(q,a, test_fn=None)

0.542

In [7]:
ds = ben.train_dataset
q,a=ds.sample_split(0, len(ds), sep='=', sep_included=-1)
ben.measure_accuracy(q,a)

0.9272222222222222

In [14]:
q=['30+16=', '91+2=', '11+11=']
a=['46','93','22']
ben.measure_accuracy(q,a)

0.6666666666666666

In [4]:
start_text='30+16=|92+17=|11+11='
ans=[]
ben.sample(start_text, dest=ans, emit_start=False)
ans

['46', '109', '22']

In [7]:
start_text='1+1='
ans=[]
ben.sample(start_text, dest=ans, emit_after='=')
ans

['2']

In [17]:
ben.state['n_samples']/128

1900.0

In [18]:
ben.train(iter_count=10000)

Training
Resuming optimizer state
Batches per epoch: 70
.CUDA max memory used: 170.00M
...................................................................................................iter 2000 (28.444 epoch): loss train=0.4626, val=0.8079, eval->0.8079
21+4=25
....................................................................................................iter 2100 (29.867 epoch): loss train=0.4588, val=0.8446, eval->0.8446
....................................................................................................iter 2200 (31.289 epoch): loss train=0.4554, val=0.9151, eval->0.9151
....................................................................................................iter 2300 (32.711 epoch): loss train=0.4531, val=0.8183, eval->0.8183
....................................................................................................iter 2400 (34.133 epoch): loss train=0.4496, val=0.8165, eval->0.8165
........................................................

In [25]:
ben = Train('add2paddrop', log_mask=LogFlag.ALL)

# set datasets
ben.set_datasets('padlinechar', train_path='../data/add2.train.txt', val_path='../data/add2.val.txt')

# set config settings
cfg = empty_config()
cfg.train.log_period=0
cfg.model.set(n_layer=6, n_head=6, n_embd=90, block_size=16, dropout=0.25)
cfg.sample.set(top=1, max_batch_size=256) # top_k(1) - always pick the best item
cfg.train.set(sample_period=-5)
cfg.trainer.set(batch_size=128)

# and init a new model with config
if ben.can_resume():
    ben.init_resume(cfg)
else:
    ben.init_new(cfg)

print(ben.get_config().dump(1))

New random seed 485812905
Initializing new model add2paddrop
Dataset train_path: ../data/add2.train.txt, val_path: ../data/add2.val.txt, train_split: None, vocab_size: 13
Model params: 0.59M
seed: 0 (int) 
sample: 
    max_len: 100 (int) 
    count: 1 (int) 
    start_text: None (NoneType) 
    start_text_sep: | (str) 
    emit_start: True (bool) 
    emit_after: None (NoneType) 
    emit_before: None (NoneType) 
    flush: True (bool) 
    eot_stop: 0 (int) 
    top: 1.0 (float) 
    temp: 1.0 (float) 
    max_batch_size: 256 (int) 
    multiline_prompt: False (bool) 
train: 
    eval_period: 100 (int) 
    eval_type: 1.0 (float) 
    eval_iters: 100 (int) 
    eval_save_checkpt: 1 (int) 
    eval_save_loss: csv,tensorboard (str) 
    sample_period: -5.0 (float) 
    log_period: 0.0 (float) 
dataset: 
    class_name: padlinechar (str) 
    train_path: ../data/add2.train.txt (str) 
    train_split: None (NoneType) 
    val_path: ../data/add2.val.txt (str) 
    params: None (NoneType) 


In [26]:
ben.train(iter_count=10000)

Training
Batches per epoch: 70
iter 0 (0.000 epoch): loss train=2.2006, val=2.2309, eval->2.2309
==> Saving model at iter=0, eval loss->2.2309 
57
.CUDA max memory used: 165.54M
...................................................................................................iter 100 (1.422 epoch): loss train=1.0424, val=1.0786, eval->1.0786
==> Saving model at iter=100, eval loss->1.0786 
....................................................................................................iter 200 (2.844 epoch): loss train=0.8437, val=0.8878, eval->0.8878
==> Saving model at iter=200, eval loss->0.8878 
....................................................................................................iter 300 (4.267 epoch): loss train=0.7831, val=0.8435, eval->0.8435
==> Saving model at iter=300, eval loss->0.8435 
....................................................................................................iter 400 (5.689 epoch): loss train=0.7550, val=0.8419, eval->0.8419
==> 

In [30]:
ds = ben.val_dataset
q,a=ds.sample_split(0, len(ds), sep='=', sep_included=-1)

def test(q,a,g):
    res = float(a == g)
    if not res:
        print(f"{q}: {a} != {g}")
    return res
    
ben.measure_accuracy(q,a, test_fn=test)

90+21=: 111 != 21
90+30=: 120 != 110
90+40=: 130 != 120
91+9=: 100 != 90
91+19=: 110 != 20
91+20=: 111 != 21
91+28=: 119 != 29
91+49=: 140 != 130
92+8=: 100 != 90
92+18=: 110 != 20
92+19=: 111 != 21
92+20=: 112 != 22
92+21=: 113 != 23
92+22=: 114 != 24
92+23=: 115 != 25
92+24=: 116 != 26
92+25=: 117 != 27
92+26=: 118 != 28
92+27=: 119 != 29
93+7=: 100 != 90
93+17=: 110 != 20
93+18=: 111 != 21
93+19=: 112 != 22
93+20=: 113 != 23
93+21=: 114 != 24
93+22=: 115 != 25
93+23=: 116 != 26
93+24=: 117 != 27
93+25=: 118 != 28
93+26=: 119 != 29
94+6=: 100 != 90
94+7=: 101 != 91
94+16=: 110 != 20
94+17=: 111 != 21
94+18=: 112 != 22
94+19=: 113 != 23
94+20=: 114 != 24
94+21=: 115 != 25
94+22=: 116 != 26
94+23=: 117 != 27
94+24=: 118 != 28
94+25=: 119 != 29
94+42=: 136 != 126
94+43=: 137 != 127
95+5=: 100 != 90
95+6=: 101 != 91
95+7=: 102 != 92
95+15=: 110 != 20
95+16=: 111 != 21
95+17=: 112 != 22
95+18=: 113 != 23
95+19=: 114 != 24
95+20=: 115 != 25
95+21=: 116 != 26
95+22=: 117 != 27
95+23=: 118 !

0.87

In [29]:
ds = ben.train_dataset
q,a=ds.sample_split(0, len(ds), sep='=', sep_included=-1)

def test(q,a,g):
    res = float(a == g)
    if not res:
        print(f"{q}: {a} != {g}")
    return res
    
ben.measure_accuracy(q,a, test_fn=test)

2+8=: 10 != 11
3+7=: 10 != 11
3+8=: 11 != 12
4+3=: 7 != 8
4+4=: 8 != 9
4+6=: 10 != 11
5+3=: 8 != 9
5+5=: 10 != 11
6+4=: 10 != 11
7+3=: 10 != 11
7+4=: 11 != 12
8+2=: 10 != 11
8+3=: 11 != 12


0.9985555555555555

In [32]:
ben = Train('add2paddrop2', log_mask=LogFlag.ALL)

# set datasets
ben.set_datasets('padlinechar', train_path='../data/add2.train.txt', val_path='../data/add2.val.txt')

# set config settings
cfg = empty_config()
cfg.train.log_period=0
cfg.model.set(n_layer=6, n_head=6, n_embd=180, block_size=16, dropout=0.25)
cfg.sample.set(top=1, max_batch_size=256) # top_k(1) - always pick the best item
cfg.train.set(sample_period=-5)
cfg.trainer.set(batch_size=128)

# and init a new model with config
if ben.can_resume():
    ben.init_resume(cfg)
else:
    ben.init_new(cfg)

print(ben.get_config().dump(1))
ben.train(iter_count=10000)

New random seed 2657223947
Initializing new model add2paddrop2
Dataset train_path: ../data/add2.train.txt, val_path: ../data/add2.val.txt, train_split: None, vocab_size: 13
Model params: 2.35M
seed: 0 (int) 
sample: 
    max_len: 100 (int) 
    count: 1 (int) 
    start_text: None (NoneType) 
    start_text_sep: | (str) 
    emit_start: True (bool) 
    emit_after: None (NoneType) 
    emit_before: None (NoneType) 
    flush: True (bool) 
    eot_stop: 0 (int) 
    top: 1.0 (float) 
    temp: 1.0 (float) 
    max_batch_size: 256 (int) 
    multiline_prompt: False (bool) 
train: 
    eval_period: 100 (int) 
    eval_type: 1.0 (float) 
    eval_iters: 100 (int) 
    eval_save_checkpt: 1 (int) 
    eval_save_loss: csv,tensorboard (str) 
    sample_period: -5.0 (float) 
    log_period: 0.0 (float) 
dataset: 
    class_name: padlinechar (str) 
    train_path: ../data/add2.train.txt (str) 
    train_split: None (NoneType) 
    val_path: ../data/add2.val.txt (str) 
    params: None (NoneType)